## Inverse Cooking: Recipe Generation from Food Images

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time

Set ```data_dir``` to the path including vocabularies and model checkpoint

In [2]:
data_dir = ''

In [3]:
# code will run in gpu if available and if the flag is set to True, else it will run on cpu
use_gpu = True
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'


# device = torch.device('cpu')
# map_loc = None


In [4]:
print(device)

cuda


In [5]:
# code below was used to save vocab files so that they can be loaded without Vocabulary class
#ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_ingrs.pkl'), 'rb'))
#ingrs_vocab = [min(w, key=len) if not isinstance(w, str) else w for w in ingrs_vocab.idx2word.values()]
#vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_toks.pkl'), 'rb')).idx2word
#pickle.dump(ingrs_vocab, open('../demo/ingr_vocab.pkl', 'wb'))
#pickle.dump(vocab, open('../demo/instr_vocab.pkl', 'wb'))

ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'ingr_vocab.pkl'), 'rb'))
vocab = pickle.load(open(os.path.join(data_dir, 'instr_vocab.pkl'), 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [73]:
def get_ingrs(ids):
    gen_ingrs = []
    for ingr_idx in ids:
        ingr_name = ingrs_vocab[ingr_idx]
        if ingr_name == '<pad>':
            print('pad!')
            break
        if ingr_name == '<end>':
            break
        gen_ingrs.append(ingr_name)
    return gen_ingrs


def mask_from_eos(ids, eos_value, mult_before=True):
    mask = torch.ones(ids.size()).to(device).byte()
    mask_aux = torch.ones(ids.size(0)).to(device).byte()

    # find eos in ingredient prediction
    for idx in range(ids.size(1)):
        # force mask to have 1s in the first position to avoid division by 0 when predictions start with eos
        if idx == 0:
            continue
        if mult_before:
            mask[:, idx] = mask[:, idx] * mask_aux
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
        else:
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
            mask[:, idx] = mask[:, idx] * mask_aux
    return mask
print (instrs_vocab_size, ingr_vocab_size)

23231 1488


In [99]:
t = time.time()
import sys; sys.argv=['']; del sys
args = get_parser()
args.maxseqlen = 15
args.ingrs_only=False
model = get_model(args, ingr_vocab_size, instrs_vocab_size)
# Load the trained model parameters
model_path = os.path.join('/home/ct2020dl5787/inversecooking/model/checkpoints', 'modelbest.ckpt')
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False
model.reduction = 'none'
print ('loaded model')
print ("Elapsed time:", time.time() -t)


loaded model
Elapsed time: 2.4150376319885254


In [65]:
ingrs_vocab[-2]

'salted_herring'

Set ```use_urls = True``` to get recipes for images in ```demo_urls```. 

You can also set ```use_urls = False``` and get recipes for images in the path in ```data_dir/test_imgs```.

In [9]:
import requests
from io import BytesIO
import random
from collections import Counter
#unpickled_df = pd.read_pickle("df_recipe1023.pkl")
use_urls = False # set to true to load images from demo_urls instead of those in test_imgs folder
show_anyways = False #if True, it will show the recipe even if it's not valid
image_folder = os.path.join('/home/ct2020dl5787/VireoFood172/sample_pictures1720')
#TODO1:load data
if not use_urls:
    demo_imgs = os.listdir(image_folder)
    random.shuffle(demo_imgs)

# demo_urls = ['https://food.fnr.sndimg.com/content/dam/images/food/fullset/2013/12/9/0/FNK_Cheesecake_s4x3.jpg.rend.hgtvcom.826.620.suffix/1387411272847.jpeg',
#          'https://www.196flavors.com/wp-content/uploads/2014/10/california-roll-3-FP.jpg']
#demo_urls = unpickled_df["url"].tolist()
demo_files = demo_urls if use_urls else demo_imgs
# demo_files = demo_files

In [72]:
demo_files = demo_files[:200]

In [10]:
transf_list_batch = []
transf_list_batch.append(transforms.ToTensor())
transf_list_batch.append(transforms.Normalize((0.485, 0.456, 0.406), 
                                              (0.229, 0.224, 0.225)))
to_input_transf = transforms.Compose(transf_list_batch)


In [73]:
len(demo_files)

200

In [11]:
import csv
vocab_file = '/home/ct2020dl5787/VireoFood172/vocab172_mapping.csv'
chinese_vocab = []
mapped_vocab = []
with open(vocab_file) as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        #check duplicate
        chinese_vocab.append(row[0])
        mapped_vocab.append(row[1])
        #print("originally: ", row[0], " mapped to ", row[1])

In [38]:
vocab = pd.read_pickle('/home/ct2020dl5787/VireoFood172/pickle_data/onehot1720matrix_vocab.pkl')
matrix = pd.read_pickle('/home/ct2020dl5787/VireoFood172/pickle_data/onehot1720matrix.pkl')
matrix_new = np.argmax(matrix,2)

In [109]:
#del model.ingredient_decoder.linear
#Linear(in_features=512, out_features=1487, bias=True)
model.ingredient_decoder.linear  = nn.Linear(in_features=512, out_features=353, bias=True)

for parma in model.image_encoder.parameters():
    parma.requires_grad = False
    
for parma in model.ingredient_decoder.layers.parameters():
    parma.requires_grad = False    

    
for parma in model.ingredient_decoder.embed_tokens.parameters():
    parma.requires_grad = False  
    
optimizer = torch.optim.Adam(model.ingredient_decoder.linear.parameters())
criterion = nn.BCELoss()


In [114]:
model.ingredient_decoder.linear
test2 = model.ingredient_decoder.sample(None,None,greedy = False,temperature=1.0,img_features=img_features, first_token_value=0,replacement=False)

RuntimeError: Expected object of type torch.cuda.FloatTensor but found type torch.FloatTensor for argument #2 'mat2'

In [80]:
new_voca = {}
new_voca['<end>'] = 0
index = 1
for token in list(vocab.keys()):
    new_voca[token] = index
    index += 1

In [98]:
model.ingredient_decoder.linear
test2 = model.ingredient_decoder.(None,None,greedy = True,temperature=1.0,
                                                           img_features=img_features, first_token_value=0,replacement=False)

AttributeError: 'DecoderTransformer' object has no attribute 'linear'

In [ ]:
forward(self, ingr_features, ingr_mask, captions, img_features, incremental_state=None):

In [58]:
##STEP1: run image
err_pic = []
img_file_list = []
start = time.time()
for img_file in demo_files:
    
    if not use_urls and img_file.endswith(".jpg"):
        image_path = os.path.join(image_folder, img_file)
        image = Image.open(image_path).convert('RGB')

        transf_list = []
        transf_list.append(transforms.Resize(256))
        transf_list.append(transforms.CenterCrop(224))
        transform = transforms.Compose(transf_list)

        image_transf = transform(image)
        image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)

#         plt.imshow(image_transf)
#         plt.axis('off')
#         plt.show()
#         plt.close()

        num_valid = 1
        img_file_list.append(img_file)
        if img_file == demo_files[0]:
            print(img_file)
            image_all = image_tensor
        else:
             image_all = torch.cat((image_all,image_tensor))


19_10_26.jpg


In [74]:
##step2: run ingredient_decoder  
## batch_size could be changed

outputs = dict()
recipt_all = []
batch_size = 100


for i in range(image_all.shape[0]//batch_size + 1):

    img_features = model.image_encoder(image_all[i*batch_size : (i+1)*batch_size])
    ingr_ids, ingr_probs = model.ingredient_decoder.sample(None,None,greedy = True,temperature=1.0,
                                                           img_features=img_features, first_token_value=0,replacement=False)
    
    sample_mask = mask_from_eos(ingr_ids, eos_value=0, mult_before=False)
    ingr_ids[sample_mask == 0] = 0

    outputs['ingr_ids'] = ingr_ids
    outputs['ingr_probs'] = ingr_probs.data

    ingr_ids = outputs['ingr_ids'].cpu().numpy()
    recipe = list(map(lambda x : get_ingrs(x), ingr_ids))
        
    recipt_all.extend(recipe)

In [36]:
params = list(model.ingredient_decoder.parameters())
optimizer = torch.optim.Adam(params, lr=args.learning_rate)
model = model.to(device)
cudnn.benchmark = True

## updata_decay_factor
# frac = epoch // args.lr_decay_every
# decay_factor = args.lr_decay_rate ** frac
# new_lr = args.learning_rate*decay_factor
# print ('Epoch %d. lr: %.5f'%(epoch, new_lr))
# set_lr(optimizer, decay_factor)

In [60]:
ingr_ids

array([[ 30,  23,  15,   9,  10,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [ 10,   9,  12, 166,  91, 109, 202,  13,   7,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [ 12,  10,   7,  91,  33, 110,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [ 12,  10,   9,  91,  20,  15,  23,   7,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  9,  12,  23,  10, 187,  30, 213,  15,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [ 10,   9,  21,  23,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [ 12, 202,  23,  82, 198,  10, 340,  91, 197,  41,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [ 10,   9,  12,  84,  91, 166,  15,  52,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  2,   9,  10,  59,  33,   7,   0,   0,   0,   0,   0,  

In [59]:
torch.max(ingr_probs[0],1)

(tensor([ 9.5538,  7.8046,  9.9353,  6.3450,  9.4968,  6.2944,  6.1642,  9.4382,
          9.9887, 10.2768, 10.3964, 10.3799, 10.2220, 10.0135,  9.7965,  9.5698,
          9.3459,  9.1348,  8.9407,  8.7568],
        device='cuda:0', grad_fn=<MaxBackward0>),
 tensor([ 15,  23,  30, 367,  12, 345, 382,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0], device='cuda:0'))

In [61]:
ingrs_vocab[15]

'water'

In [42]:
true_label_df = pd.read_pickle('/home/ct2020dl5787/VireoFood172/pickle_data/sample1720.pkl')

In [44]:
true_label_df['true_ingredients']

0             [scallion, pepper, beef]
1             [scallion, pepper, beef]
2             [scallion, pepper, beef]
3                      [Parsley, beef]
4       [Parsley, chili, garlic, beef]
                     ...              
1715                     [tofu, chili]
1716           [scallion, tofu, chili]
1717     [scallion, pork, tofu, chili]
1718          [scallion, pepper, tofu]
1719           [scallion, tofu, chili]
Name: true_ingredients, Length: 1720, dtype: object

In [23]:
generate_data = pd.DataFrame([img_file_list,recipt_all]).T
generate_data = generate_data.rename(columns={0: "img_names", 1: 'generate_ingre'})

In [25]:
all_info = pd.merge(true_label_df,generate_data, how='inner', on=['img_names'])
all_info.to_pickle('output/ingr_only_1720.pkl')

,img_names,true_ingredients,ingredients_Chinese,generate_ingre
0,92_075811n2tr1trtvr21vvao.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[pepper, oil, onion, clove, soy_sauce, sugar, ..."
1,92_075817xlk22xxxagbk296x.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[pepper, oil, onion, clove, beans, salt, tomato]"
2,92_100053030.1.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[pepper, onion, oil, chicken, mushroom, salt, ..."
3,92_10_0.jpg,"[Parsley, beef]","[Chinese Parsleycoriander, Shredded beef tripe]","[pepper, oil, salt, potato, clove, onion, pars..."
4,92_10_1.jpg,"[Parsley, chili, garlic, beef]","[Chinese Parsleycoriander, Crushed hot and dry...","[oil, onion, pepper, soy_sauce, sugar, chicken..."
...,...,...,...,...
1715,63_10_14.jpg,"[tofu, chili]","[Tofu chunks, Chili oil]","[onion, pepper, oil, chicken, clove, tomato, w..."
1716,63_10_16.jpg,"[scallion, tofu, chili]","[Minced green onion, Tofu chunks, Chili oil]","[onion, pepper, oil, soy_sauce, clove, broth, ..."
1717,63_10_18.jpg,"[scallion, pork, tofu, chili]","[Minced green onion, Minced pork, Tofu chunks,...","[onion, pepper, potato, clove, oil, tomato, sa..."
1718,63_10_19.jpg,"[scallion, pepper, tofu]","[Minced green onion, Hot and dry pepper, Tofu ...","[onion, oil, soy_sauce, pepper, chicken, water..."


In [26]:
all_info[:50]

,img_names,true_ingredients,ingredients_Chinese,generate_ingre
0,92_075811n2tr1trtvr21vvao.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[pepper, oil, onion, clove, soy_sauce, sugar, ..."
1,92_075817xlk22xxxagbk296x.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[pepper, oil, onion, clove, beans, salt, tomato]"
2,92_100053030.1.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[pepper, onion, oil, chicken, mushroom, salt, ..."
3,92_10_0.jpg,"[Parsley, beef]","[Chinese Parsleycoriander, Shredded beef tripe]","[pepper, oil, salt, potato, clove, onion, pars..."
4,92_10_1.jpg,"[Parsley, chili, garlic, beef]","[Chinese Parsleycoriander, Crushed hot and dry...","[oil, onion, pepper, soy_sauce, sugar, chicken..."
5,92_10_10.jpg,"[Parsley, cucumber, beef]","[Chinese Parsleycoriander, Julienned cucumber,...","[pepper, oil, onion, salt, soy_sauce, clove]"
6,92_10_11.jpg,"[pepper, beef]","[Pepper slices, Shredded beef tripe]","[oil, pepper, onion, clove, soy_sauce, ginger,..."
7,92_10_16.jpg,"[Parsley, beef]","[Chinese Parsleycoriander, Shredded beef tripe]","[onion, soy_sauce, sugar, sake, water, mirin, ..."
8,92_10_17.jpg,"[diced_green_chilis, beef, celery]","[Crushed pepper, Shredded beef tripe, Celery]","[oil, pepper, soy_sauce, clove, onion, ginger,..."
9,92_10_24.jpg,"[diced_green_chilis, beef, celery]","[Crushed pepper, Shredded beef tripe, Celery]","[oil, pepper, soy_sauce, clove, onion, sugar, ..."


In [16]:
##step3:save info
generate_data = pd.DataFrame([img_file_list,recipt_all]).T
generate_data = generate_data.rename(columns={0: "url", 1: "generate_ingre"})
all_info = pd.merge(unpickled_df, generate_data , how='inner', on=['url'])
all_info.to_pickle('output/recipt_only.pkl')
all_info.head()

,id,ingredients,instructions,url,generate_ingre


In [ ]:
##step4:evalutation
def get_ingred_f1(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        F-1 score of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.4
    '''
    intersection = list(set(pred) & set(label))
    precision = len(intersection) / len(pred)
    recall = len(intersection) / len(label)
    f1 = 2 * precision * recall / (precision + recall)
    return f1


def get_ingred_IOU(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        IOU of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.25
    '''
    intersection = len(list(set(pred) & set(label)))
    union = len(list(set(pred) | set(label)))
    iou = intersection / union
    return iou


import nltk
import string
def get_bleu_n_score(pred, label, n = 4):
    
    '''
    TODO: STEM not added
    input: 
        pred: One string of predict recipe 
        label: One string of reference recipe
        n(optional): up to n-gram.
    output: 
        bleu score
    
    i.e.  
    in: 
        pred = "Add the buttter"
        label = "Add half butter and mix well"
    out: 
        0.25
    '''
    weights = [1/n] * n
    pred_list = pred.translate(str.maketrans('', '', string.punctuation)).split()
    label_list = label.translate(str.maketrans('', '', string.punctuation)).split()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([label_list], pred_list, weights)
    return BLEUscore



from nltk.translate.meteor_score import meteor_score
def get_meteor_score(pred, label):
    return meteor_score([label], pred)